# Collect Datasets
First we look at the discipline page.

## Expulsion

In [40]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import os

In [15]:
url = "https://www.cde.ca.gov/ds/ad/filesed.asp"
html = requests.get(url).text
soup = BeautifulSoup(html,'html')

In [22]:
table = soup.find("table")
files = {}
for j,row in enumerate(table.find_all("tr")):
    if j==0: continue #Skip header of the table
    date = None
    file_path = None
    for i,cell in enumerate(row.find_all("td")):
        if i==0:
            #Get first column as the date
            date = cell.text.strip()
        elif i==1:
            file_path = cell.find('a').attrs['href']
    files[date]=file_path
    print(date,file_path)

2020–21 https://www3.cde.ca.gov/demo-downloads/discipline/susp2021.txt
2019–20 https://www3.cde.ca.gov/demo-downloads/discipline/susp1920.txt
2018–19 https://www3.cde.ca.gov/demo-downloads/discipline/susp1819.txt
2017–18 https://www3.cde.ca.gov/demo-downloads/discipline/susp1718.txt
2016–17 https://www3.cde.ca.gov/demo-downloads/discipline/susp1617.txt
2015–16 https://www3.cde.ca.gov/demo-downloads/discipline/susp1516.txt
2014–15 https://www3.cde.ca.gov/demo-downloads/discipline/susp1415.txt
2013–14 https://www3.cde.ca.gov/demo-downloads/discipline/susp1314.txt
2012–13 https://www3.cde.ca.gov/demo-downloads/discipline/susp1213.txt
2011–12 https://www3.cde.ca.gov/demo-downloads/discipline/susp1112.txt


Now download them all

use pandas to put them into dataframes

In [29]:
texts = [requests.get(file_path).text for _,file_path in files.items()]

Turn them to dataframes, I have to build the strucutre myself though... parsing.

In [126]:
def clean_line(line):
    return line.replace("ï»¿","").strip()

In [139]:
#Given the text from the file
#Return an an array of arrays that pandas can read. 
def clean(string):
    lines = string.split('\n')
    return [clean_line(line).split('\t') for line in lines]

In [140]:
dataframes = []
for text in texts:
    data = clean(text)
    dataframes.append(pd.DataFrame(data[1:],columns=data[0]))

In [141]:
df = pd.concat(dataframes)

,AcademicYear,AggregateLevel,CountyCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,ReportingCategory,...,Unduplicated Count of Students Suspended (Total),Unduplicated Count of Students Suspended (Defiance-Only),Suspension Rate (Total),Suspension Count Violent Incident (Injury),Suspension Count Violent Incident (No Injury),Suspension Count Weapons Possession,Suspension Count Illicit Drug-Related,Suspension Count Defiance-Only,Suspension Count Other Reasons,Cumulative Enrollment
0,2020-21,C,01,,,Alameda,,,All,GF,...,18,4,0.0,0,11,1,1,4,1,NaN
1,2020-21,C,01,,,Alameda,,,All,GM,...,87,13,0.1,11,52,2,3,13,11,NaN
2,2020-21,C,01,,,Alameda,,,All,GX,...,0,0,0.0,0,0,0,0,0,0,NaN
3,2020-21,C,01,,,Alameda,,,All,RA,...,30,6,0.0,1,18,0,0,6,6,NaN
4,2020-21,C,01,,,Alameda,,,All,RB,...,6,1,0.0,0,5,0,0,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345156,2011-12,T,00,,,State,,,No,SS,...,267665,131270,7.4,39743,168137,11589,38440,259383,26603,3611833
345157,2011-12,T,00,,,State,,,No,TA,...,352429,170062,6.0,50553,208868,15014,53127,324745,33503,5890875
345158,2011-12,T,00,,,State,,,All,TA,...,365743,176027,5.8,52086,217052,15446,55369,335079,34670,6285600
345159,2011-12,T,00,,,State,,,Yes,TA,...,14066,6180,2.9,1533,8184,432,2242,10334,1167,490897


In [142]:
df.to_csv("expulsion.csv")
df